In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/10 22:39:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
pyspark.__version__

'3.3.2'

In [13]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv('fhvhv_tripdata_2021-06.csv')

In [15]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [19]:
from pyspark.sql import types

In [20]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [21]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv')

In [23]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [24]:
df \
    .repartition(12) \
    .write.parquet("data/pq/fhvhv/2021/06/")

In [26]:
from pyspark.sql import functions as F

In [27]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

452470

In [29]:
df.registerTempTable('fhvhv_2021_06')

/home/albert_tests/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [31]:
spark.sql("""
SELECT
    COUNT(*)
FROM
    fhvhv_2021_06
WHERE
    to_date(pickup_datetime) = '2021-06-15';
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [52]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('duration_h', 
                F.round((df.dropoff_datetime.cast("long") - df.pickup_datetime.cast("long")) / 3600, 2)
        ) \
    .groupBy('pickup_date') \
    .max('duration_h') \
    .orderBy(F.col('max(duration_h)').desc()) \
    .limit(5) \
    .show()

+-----------+---------------+
|pickup_date|max(duration_h)|
+-----------+---------------+
| 2021-06-25|          66.88|
| 2021-06-22|          25.55|
| 2021-06-27|          19.98|
| 2021-06-26|           18.2|
| 2021-06-23|          16.47|
+-----------+---------------+



In [60]:
spark.sql("""
SELECT
    to_date(pickup_datetime),
    MAX(ROUND((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600, 2)) AS max_duration_h
FROM
    fhvhv_2021_06
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT
    10
;
""").show()

+------------------------+--------------+
|to_date(pickup_datetime)|max_duration_h|
+------------------------+--------------+
|              2021-06-25|         66.88|
|              2021-06-22|         25.55|
|              2021-06-27|         19.98|
|              2021-06-26|          18.2|
|              2021-06-23|         16.47|
|              2021-06-24|         13.91|
|              2021-06-04|         11.67|
|              2021-06-20|         10.98|
|              2021-06-01|         10.27|
|              2021-06-28|          9.97|
+------------------------+--------------+



In [61]:
df_zones = spark.read.parquet('zones/')

In [66]:
df \
    .join(df_zones, df.PULocationID == df_zones.LocationID) \
    .groupBy(df_zones.LocationID) \
    .count() \
    .orderBy(F.col('count()')) \
    .show()

AnalysisException: Column '`count()`' does not exist. Did you mean one of the following? [count, LocationID];
'Sort ['count() ASC NULLS FIRST], true
+- Aggregate [LocationID#1007], [LocationID#1007, count(1) AS count#1247L]
   +- Join Inner, (PULocationID#139 = cast(LocationID#1007 as int))
      :- Relation [dispatching_base_num#136,pickup_datetime#137,dropoff_datetime#138,PULocationID#139,DOLocationID#140,SR_Flag#141,Affiliated_base_number#142] csv
      +- Relation [LocationID#1007,Borough#1008,Zone#1009,service_zone#1010] parquet
